In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# read the data
train_data = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test_data = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

# remove rows with missing target, separate target from predictors
train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = train_data.SalePrice
train_data.drop(['Id', 'SalePrice'], axis=1, inplace=True)
test_data.drop(['Id'], axis=1, inplace=True)

# to keep things simple, we'll use only numeric predictors
X = train_data.select_dtypes(exclude=['object'])
X_test = test_data.select_dtypes(exclude=['object'])

# break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3,
                                                      random_state=0)

In [ ]:
X_train.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [ ]:
# get names of columns with missing values
# (X contains all of the rows in both X_train and X_valid)
cols_with_missing = [col for col in X.columns 
                                 if X[col].isnull().any()]

# drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

In [ ]:
from sklearn.impute import SimpleImputer

# imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

In [ ]:
# make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

In [ ]:
# print shape of dataset (num_rows, num_columns)
print(X.shape)

# print number of missing values in each column of training data
missing_val_count_by_column = (X.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

### imputation + encoding

In [ ]:
# "cardinality" means the number of unique values in a column
# select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in train_data.columns if
                        train_data[cname].nunique() < 10 and
                        train_data[cname].dtype == "object"]

# select numeric columns
numeric_cols = [cname for cname in train_data.columns if
                train_data[cname].dtype in ['int64', 'float64']]

# keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X = train_data[my_cols]
X_test_full = test_data[my_cols]

# break off validation set from training data
X_train_full, X_valid_full, y_train_full, y_valid_full = train_test_split(X, y, 
                                                                          train_size=0.7, test_size=0.3,
                                                                          random_state=0)

In [ ]:
X_train_full.head()

In [ ]:
def score_dataset_two(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=500, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [ ]:
# make copy to avoid changing original data (when imputing)
X_train_plus_full = X_train_full.copy()
X_valid_plus_full = X_valid_full.copy()

# make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus_full[col + '_was_missing'] = X_train_plus_full[col].isnull()
    X_valid_plus_full[col + '_was_missing'] = X_valid_plus_full[col].isnull()
    
# one-hot encode categorical data
OH_X_train = pd.get_dummies(X_train_plus_full)
OH_X_valid = pd.get_dummies(X_valid_plus_full)

# ensure columns are in same order in both datasets
OH_X_train, OH_X_valid = OH_X_train.align(OH_X_valid, join='inner', axis=1)

# imputation
my_imputer = SimpleImputer()
imputed_OH_X_train = pd.DataFrame(my_imputer.fit_transform(OH_X_train))
imputed_OH_X_valid = pd.DataFrame(my_imputer.transform(OH_X_valid))

# imputation removed column names; put them back
imputed_OH_X_train.columns = OH_X_train.columns
imputed_OH_X_valid.columns = OH_X_valid.columns

print("MAE:") 
print(score_dataset_two(imputed_OH_X_train, imputed_OH_X_valid, y_train, y_valid))

imputed_OH_X_train.head()